# Coursework 3: Reed-Solomon Codes

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

**Julia Packages**

We use the `LinearAlgebra` package for matrix operations.
We use the `StatsBase` package for introducing random error.

In [1]:
using Pkg
Pkg.add("LinearAlgebra")
Pkg.add("StatsBase")

using LinearAlgebra
using StatsBase

    Updating registry at `C:\Users\aydes\.julia\registries\General`

#### Helper Functions

In [47]:
# Helper Functions

# Function to do modulo division
function modulo_div(num,den,p)
    return mod((num * gcdx(den,p)[2]),p)
end

# Function for modular exoponentiation
function modExpo(p,x,n)
    if p == 1 
        return 0
    end
    a = 1 # initialise a
    binaryExp = mod(x,p) #binaryExp when i=0
    while n > 0 #loop cycles through each binary digit of n - (cycle of ceiling[log2(n)] times)
        if (mod(n,2) == 1) #check LSB
            a = mod((a * binaryExp), p) #if LSB is 1 then update a accordingly
        end
        n = n >> 1 #right shift exponent to check next bit
        binaryExp = mod((binaryExp * binaryExp), p) #calculate new binaryExp value by squaring previous result and taking modulus
    end
    return a
end

modExpo (generic function with 1 method)

In [72]:
#import Base.+
struct MyPolynomial
    coeffs::Vector{Any}
    order::Int
    p::Int
    
    function MyPolynomial(coeffs,p)::MyPolynomial
        coeffs = [mod(i,p) for i in coeffs]
        return new(coeffs,length(coeffs) - 1,p)
    end
end

function remove_top_zeros(poly::MyPolynomial)
    coeffs = poly.coeffs
    while length(coeffs) > 0 && last(coeffs) == 0
        pop!(coeffs)
    end
    if length(coeffs) == 0
        push!(coeffs,0)
    end
    return MyPolynomial(coeffs,poly.p)
end

function make_same_length(poly1::MyPolynomial, poly2::MyPolynomial)
    len1 = length(poly1.coeffs)
    len2 = length(poly2.coeffs)

    if len1 > len2
        zero_pad = zeros(Int,len1-len2)
        new_coeffs = cat(poly2.coeffs,zero_pad,dims=1)
        new_poly = MyPolynomial(new_coeffs,poly2.p)
        return (poly1,new_poly)
    else
        zero_pad = zeros(Int,len2-len1)
        new_coeffs = cat(poly1.coeffs,zero_pad,dims=1)
        new_poly = MyPolynomial(new_coeffs,poly1.p)
        return (poly2,new_poly)

    end
end

function Base.:length(poly::MyPolynomial)
    return length(poly.coeffs)
end

function Base.:+(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    poly1,poly2 = make_same_length(poly1,poly2)

    new_coeffs = poly1.coeffs + poly2.coeffs
    return remove_top_zeros(MyPolynomial(new_coeffs,p))
end

function Base.:-(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    poly1,poly2 = make_same_length(poly1,poly2)
    new_coeffs = poly2.coeffs - poly1.coeffs
    return remove_top_zeros(MyPolynomial(new_coeffs,p))
end

function Base.:*(poly::MyPolynomial, num::Int)
    p = poly.p
    new_coeffs = [mod(i,p) for i in (poly.coeffs * num)]
    return remove_top_zeros(MyPolynomial(new_coeffs,p))
end

function Base.:*(num::Int, poly::MyPolynomial)
    return poly * num
end

function Base.:*(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    c1 = poly1.coeffs
    c2 = poly2.coeffs
    m = length(c1)
    n = length(c2)

    new_coeffs = zeros(Int,m+n-1)
    for i in 1:m
        for j in 1:n
            new_coeffs[i+j-1] += c1[i]*c2[j]
        end
    end

    return remove_top_zeros(MyPolynomial(new_coeffs,p))
end

function Base.:(==)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs == poly2.coeffs
end

function Base.:(!=)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs != poly2.coeffs
end

function Base.:divrem(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    num = poly1.coeffs
    den = poly2.coeffs

    if length(num) >= length(den)
        # Shift denominator to the right to match degree of numerator.
        shiftamount = length(num) - length(den)
        den = append!(zeros(Int,shiftamount),den)
    else
        # This is the case where degree of denominator > degree of numerator.
        # Results in zero quotient and the numerator being the remainder.
        return (remove_top_zeros(MyPolynomial(num,p)),MyPolynomial([0],p))
    end

    quotient = []
    divisor = last(den)

    for i in 1:(shiftamount+1)
        #Get the next coefficient of the quotient.
        mult = modulo_div(last(num),divisor,p)
        quotient = append!([mult],quotient)
        
        if mult != 0
            d = [mult * u for u in den]
            for i in 1:length(num)
                num[i] = mod((num[i] - d[i]),p)
            end
        end
        
        pop!(num)
        popfirst!(den)
    end

    return (remove_top_zeros(MyPolynomial(num,p)),remove_top_zeros(MyPolynomial(quotient,p)))

end

function Base.:gcdx(poly1::MyPolynomial,poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    if length(poly1) >= length(poly2)
        swapped = false
        a = poly1
        b = poly2
    else
        swapped = true
        a = poly2
        b = poly1
    end

    x0,x1 = MyPolynomial([1],p),MyPolynomial([0],p)
    y0,y1 = MyPolynomial([0],p),MyPolynomial([1],p)

    if b == MyPolynomial([0],p)
        if swapped
            return (a,y0,x0)
        else
            return (a,x0,y0)
        end
    end

    r = b

    while r != MyPolynomial([0],p)
        r,q = divrem(a,b)
        a = b
        b = r

        x0, x1 = x1, x0 - q*x1
        y0, y1 = y1, y0 - q*y1
    end

    if swapped
        return (a,y0,x0)
    else
        return (a,x0,y0)
    end
end


## 3.1 RS Codes on $\mathbb{F}_p$ (50%)

### 3.1.1 RS Codes on $\mathbb{F}_{17}$

1. In $\mathbb{F}_{17}$, find a primitive element $\alpha$. 
2. Let $\mathcal{C} \subset \mathbb{F}_{17}^{16}$ be an RS code with parameters $[16,k,7]$. What is the value of $k$? 
3. Design and implement an encoding function `RSNum_Enc` that encodes a message $\boldsymbol{m} \in \mathbb{F}_{17}^k$ to a codeword $\boldsymbol{c} \in \mathcal{C}$. 
4. Let $\boldsymbol{y} = \boldsymbol{c} + \boldsymbol{e}$ be the received word where the channel introduces $t = \lfloor \frac{d-1}{2}\rfloor$ errors in random places. Design and implement a function `RSNum_Channel` for this. Julia function `StatsBase > Sample` can be helpful in randomly chose error locations.
5. Design and implement a function `RSNum_Syndrome` to calculate the syndrome vector and the syndrome polynomial from the received word $\boldsymbol{y}$. 
6. Design and implement a function `RSNum_KeyPolys` to find error locator polynomial $L(z)$ and error evaluator polynomial $E(z)$ from the syndrome polynomial $S(z)$. 
7. Design and implement a function `RSNum_Err` to find the estimated error vector $\hat{\boldsymbol{e}}$ and the estimated codeword $\hat{\boldsymbol{c}}$.
8. Design and implement a function `RSNum_Message` to find the estimated message $\hat{\boldsymbol{m}}$. 
9. Implement a function `RSNum_Dec` which uses the functions in Steps 5-8 as sub-routines for decoding.  

Provide necessary documentation.

#### 1. Find a Primitive Element $\alpha$

We use the function `find_primitve(p)`, which returns an array of all primitive elements in the finite field. We then pick one at random to be our $\alpha$.

In [81]:
function is_primitive_element(element,p)
    if modExpo(p,element,p-1) != 1
        return false
    end

    # Using ord(β) | p-1 for Fₚ.
    # Using p instead of q as p = q for integer ring finite fields.
    # We only need to check powers that are factors of p-1.
    factors = []
    for i in 2:isqrt(p-1)
        quot,rem = divrem(p-1,i)
        if rem == 0
            append!(factors,[i,quot])
        end
    end
    
    for t in factors
        if modExpo(p,element,t) == 1
            return false
        end
    end
    return true
end

# Find primitive elements for finite field Fₚ
function find_primitive(p)
    field = collect(0:p-1)
    prim_els = []
    for a in field
        if is_primitive_element(a,p)
            push!(prim_els,a)
        end
    end
    return prim_els
end

find_primitive(17)

8-element Vector{Any}:
  3
  5
  6
  7
 10
 11
 12
 14

In [82]:
# Define the prime number for our Finite Field
p = 17

# Get all primitive elements in that Finite Field
prim_els = find_primitive(p)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
@show α

α = 11


11

#### 2. What is the value of $k$?

RS Codes are defined by three parameters: **$[n,k,d]$**, where $d = n - k + 1$.

Taking the RS Code given in the question:  **$[16,k,7]$**, we can see that $n = 16$ and $n - k + 1 = 7$. 

We can substitute in $n = 16$ to get: $16 - k + 1 = 7$. Solving this equation yields **$k = 10$**.

In [83]:
# Define the values of [n,k,d]
n = 16
k = 10
d = 7

7

#### 3. RS Encoding

In [84]:
# Helper Functions for RS Encoding

"""
generator_matrix(p,α,n,k)

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A k×n matrix of type Matrix{Int} which is the generator matrix of the RS Code [n,k,d]
"""
function generator_matrix(p,α,n,k)
    # First row of the matrix will always be n 1s, so generate that
    gen_mat = ones(Int,1,n)
    
    for i in 1:k-1
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Mulitply the defining element for the given row by ascending powers up to n
            row[j] = modExpo(p,row[j],j-1)
        end
        # Add the new row to the bottom of the matrix
        gen_mat = vcat(gen_mat,row)
    end
    return gen_mat
end


generator_matrix

In [85]:
"""
RSNum_Enc(message,p,α,n,k)

Args:
message: Matrix{Int} of dimension 1×k containing the message
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A 1×n matrix of type Matrix{Int}, which is the encoded message
"""

function RSNum_Enc(message,p,α,n,k)
    if typeof(message) != Matrix{Int}
        println("Please enter message in the format of a 1×k matrix")
        println("Type of message should be Matrix{Int}, not $(typeof(message))")
        return -1
    end
    if length(message) != k
        println("Length of message should be $k, which is the value of k provided")
        return -1
    end

    # Ensure each element in our message is in Fₚ
    # If not then, perform modulo to make it comply
    message = [mod(i,p) for i in message]

    # Calculate the Generator Matrix for the given parameters
    G = generator_matrix(p,α,n,k)

    # Calculate the code by multiplying the message and the generator matrix
    code = message * G
    code = [mod(i,p) for i in code]

    return code

end

RSNum_Enc (generic function with 1 method)

In [86]:
# Test: Encode a message
message = [7 3 8 7 1 9 4 4 0 0]
codeword = RSNum_Enc(message,p,α,n,k)

1×16 Matrix{Int64}:
 9  8  0  9  0  12  16  14  14  8  10  5  9  16  15  1

#### Simulating the Channel

In [87]:
"""
RSNum_Channel(code,p,n,d)

Args:
code: Matrix{Int} of dimension 1×n, the codeword to be transmitted accross the channel
p: The prime number that defines the Finite Field Fₚ
n: The length of the code
d: d as defined for the RS Code

Returns:
A 1×n matrix of type Matrix{Int}, which is the codeword with error introduced
"""

function RSNum_Channel(code,p,n,d)
    # Maximum number of errors
    t::Int = floor((d+1)/2)

    # Randomly choose t unique locations of the errors
    I = sample((1:n),t,replace=false)
    @show I
    # Build error vector, which has non-zero elements at eᵢ where i∈I
    e = []
    for i in 1:n
        if i in I
            push!(e,rand(1:p))
        else
            push!(e,0)
        end
    end

    # Convert e from type Vector{Int} to Matrix{Int}
    # This makes it the same type and shape as the code, allowing for addition
    e = reshape(e,1,n)
    @show e
    # Add error to the code
    changed_code = code + e
    changed_code = [mod(i,p) for i in changed_code]

    return changed_code
end


RSNum_Channel (generic function with 1 method)

In [88]:
# Test: Transmit codeword through channel
received = RSNum_Channel(codeword,p,n,d)

I = [7, 14, 15, 9]
e = 

Any[0 0 0 0 0 0 8 0 14 0 0 0 0 15 10 0]


1×16 Matrix{Int64}:
 9  8  0  9  0  12  7  14  11  8  10  5  9  14  8  1

#### Syndrome Vector and Polynomial

In [89]:
# Helper functions

"""
parity_check_matrix(p,α,n,k)

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A (n-k)×n matrix of type Matrix{Int} which is the parity-check matrix of the RS Code [n,k,d]

"""

function parity_check_matrix(p,α,n,k)
    pc_mat = zeros(Int,n-k,n)
    for i in 1:n-k
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Multiply the defining element for the given row by ascending powers up to n
            pc_mat[i+(j-1)*(n-k)] = modExpo(p,row[j],j-1)
        end
    end
    return pc_mat
end

parity_check_matrix(p,α,n,k)


6×16 Matrix{Int64}:
 1  11   2   5   4  10   8   3  16   6  15  12  13   7   9  14
 1   2   4   8  16  15  13   9   1   2   4   8  16  15  13   9
 1   5   8   6  13  14   2  10  16  12   9  11   4   3  15   7
 1   4  16  13   1   4  16  13   1   4  16  13   1   4  16  13
 1  10  15  14   4   6   9   5  16   7   2   3  13  11   8  12
 1   8  13   2  16   9   4  15   1   8  13   2  16   9   4  15

In [92]:
G = generator_matrix(p,α,n,k)
H = parity_check_matrix(p,α,n,k)

A = G * transpose(H)
A = [mod(i,p) for i in A]

10×6 Matrix{Int64}:
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0

In [90]:
function RSNum_Syndrome(changed_code,p,α,n,k)
    # Calculate the parity-check Matrix
    H = parity_check_matrix(p,α,n,k)

    # Calculate syndrome vector
    syndrome_vector = H * transpose(changed_code)
    syndrome_vector = [mod(i,p) for i in syndrome_vector]

    # Return the syndrome vector
    # Note: As our convention for polynomials is that they are represented by an array of their coefficients,
    # in ascending order of the power of the term they multiply, THE SYNDROME VECTOR REPRESENTS THE SYNDROME POLYNOMIAL
    # Convention example: 5z³ + 4z² + 1 --> [1, 0, 4, 5].
    return syndrome_vector
end

RSNum_Syndrome (generic function with 1 method)

In [91]:
# Test: Calculate Syndrome Vector/Polynomial from Received Codeword
RSNum_Syndrome(received,p,α,n,k)

6×1 Matrix{Int64}:
  7
 14
 10
  5
 14
  0

#### Error Locator Polynomial and Error Evaluator Polynomial

### 3.1.2 RS Codes on $\mathbb{F}_{p}$

The above code should work well for other choices of $p$, $\alpha$, $[n,k,d]$. Try another $p$ with $30<p<100$ and demonstrate the correctness of your code. 

## 3.2 RS Codes on $\mathbb{F}_{p^k}$ (50%)

### 3.2.1 RS Codes on $\mathbb{F}_{16}$

Now we consider the finite field $\mathbb{F}_{16} = \mathbb{F}_2[z]/z^4 + z +1$, and an RS code $\mathcal{C} \subset \mathbb{F}_{16}$ with parameters $[15,k,7]$. 

Repeat the steps in 3.1.1 and change the function names from `RSNum_...` to `RSPoly_...`. Provide necessary documentation. 

### 3.2.2 RS Codes on $\mathbb{F}_{2^k}$

The above code should work for other choices of $k$ in $\mathbb{F}_{2^k}$. In 3.2.1, we have worked on the case where $k=4$. Try a different $k$ with $5 \le k \le 10$. 

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.